### Import Required Packages

In [1]:
# Imports
import os
import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import chain
from random import randint #for random colour generation
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

### Input Data From User 

In [2]:
#Market analysed: 'Investment','FullYear','DayAhead','Balancing' (choose one or several)
markets=['DayAhead'] 
input_price = 'PriceElectricityHourly'
input_prod = 'ProductionHourly'
output = 'WS_correlations'

WS_file1 = '2020_REF_WSTimeseries_to_Balmorel'
WS_file2 = '2040_REF_WSTimeseries_to_Balmorel'
WS_file3 = '2050_LW_WSTimeseries_to_Balmorel'

first_timestep="2012-01-02"
#Meaning of SSS and TTT in the data: 'DaysHours','Hours5min','WeeksHours'
meaning_SSS_TTT='DaysHours'
#Time size of each time step in TTT for creating timestamp
size_timestep="3600s"

regions = 'all' # Choose list of regions (e.g. ['BE', 'NL', 'DK1'] or 'all' (i.e. all regions that produce on-shore wind) )

color = ['#06266F',  '#a64b00', '#006363', '#a66f00']

### Plot Settings

In [3]:
# Set plotting specifications
%matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# Read Input Files

#### Price and Wind Production

In [4]:
data=pd.DataFrame()
price_data=pd.DataFrame()
windgen_ons = pd.DataFrame()
for market in markets:
    csvfiles = []
    for file in glob.glob("./input/results/" + market + "/*.csv"):
        csvfiles.append(file)

    csvfiles=[file.replace('./input\\','') for file in csvfiles] 
    csvfiles=[file.replace('.csv','') for file in csvfiles]  
    csvfiles=[file.split('_') for file in csvfiles]  
    csvfiles = np.asarray(csvfiles)  
    csvfiles=pd.DataFrame.from_records(csvfiles)
    
    csvfiles.rename(columns={0: 'Output', 1: 'Scenario',2: 'Year',3:'Subset'}, inplace=True)
    scenarios=csvfiles.Scenario.unique().tolist()
    years=csvfiles.Year.unique().tolist()
    subsets=csvfiles.Subset.unique().tolist()

    for scenario in scenarios:
        for year in years:
            for subset in subsets:
                price_file = "./input/results/"+ market + "/" + input_price + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(price_file):
                    df=pd.read_csv(price_file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    df.rename(columns = {'Val' : 'Price'}, inplace = True)
                    price_data=price_data.append(df)
                prod_file = "./input/results/"+ market + "/" + input_prod + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(prod_file):
                    df=pd.read_csv(prod_file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    windgen_ons=windgen_ons.append(df)
windgen_ons = windgen_ons.loc[windgen_ons['TECH_TYPE'] == 'WIND-ON', ]

#### Wind Speeds and Region-Area file

In [5]:
df_ws_2020 = pd.read_csv('./input/' + WS_file1 + '.csv', usecols = lambda col: col not in ['Unnamed: 0'])
df_ws_2040 = pd.read_csv('./input/' + WS_file2 + '.csv', usecols = lambda col: col not in ['Unnamed: 0'])
df_ws_LW = pd.read_csv('./input/' + WS_file3 + '.csv', usecols = lambda col: col not in ['Unnamed: 0'])
windspeeds = pd.concat([df_ws_2020, df_ws_2040, df_ws_LW], axis = 1)

RRR_AAA = pd.read_csv('./input/RRRAAA.csv') #Dictionary telling to which region each area belongs

# Processing Data Files

#### Select Regions

In [6]:
if regions == 'all':
    regions = np.array(windgen_ons.RRR.unique() ) #Creates an array of all regions where electricity is generated by on-shore wind turbines

#Price + Production
price_data = price_data.loc[price_data['RRR'].isin(regions), ]                    
windgen_ons = windgen_ons.loc[windgen_ons['RRR'].isin(regions), ]

#Wind Speeds
windspeeds = pd.DataFrame(windspeeds.stack()).reset_index()
windspeeds.columns = ['hour', 'AAA', 'WS']
windspeeds = pd.merge(windspeeds, RRR_AAA, on = 'AAA', left_index = True)
windspeeds = windspeeds.loc[windspeeds['RRR'].isin(regions), ]

#### Timestamp addition

In [7]:
full_timesteps = pd.read_csv('./input/full_timesteps_'+meaning_SSS_TTT+'.csv')
full_timesteps['Key']=full_timesteps['SSS']+full_timesteps['TTT']
number_periods=len(full_timesteps.Key.unique())
full_timesteps['timestamp']= pd.date_range(first_timestep, periods = number_periods, freq =size_timestep)
dict_timestamp=dict(zip(full_timesteps.Key, full_timesteps.timestamp))

price_data.loc[:,'timestamp']=price_data.loc[:, 'SSS']+price_data.loc[:,'TTT']
price_data.loc[:,'timestamp']=price_data.loc[:,'timestamp'].map(dict_timestamp)
windgen_ons.loc[:,'timestamp']=windgen_ons.loc[:,'SSS']+windgen_ons.loc[:,'TTT']
windgen_ons.loc[:,'timestamp']=windgen_ons.loc[:,'timestamp'].map(dict_timestamp)


In [8]:
full_timesteps = full_timesteps.reset_index()
full_timesteps = full_timesteps.rename(columns = {'index' : 'hour'})
windspeeds = pd.merge(windspeeds, full_timesteps[['hour', 'timestamp']], on = 'hour')

#### Replace possible "Eps" with 0

In [9]:
windgen_ons = windgen_ons.copy()
windgen_ons.Val=windgen_ons.Val.replace('Eps', 0)
windgen_ons.Val=pd.to_numeric(windgen_ons.Val)
price_data.Price=price_data.Price.replace('Eps', 0)
price_data.Price=pd.to_numeric(price_data.Price)

#### Change names

In [10]:
#Only LW wind speed data for 2050 is available: all LW areas get 2050 wind speeds
windgen_ons.loc[windgen_ons['AAA'].str.contains('LW') == True, 'AAA'] = windgen_ons['AAA'].str.replace('20-30', '50')
windgen_ons.loc[windgen_ons['AAA'].str.contains('LW') == True, 'AAA'] = windgen_ons['AAA'].str.replace('30-40', '50')
windgen_ons.loc[windgen_ons['AAA'].str.contains('LW') == True, 'AAA'] = windgen_ons['AAA'].str.replace('40-50', '50')

#Only REF wind speed data for 20-30 and 40-50 are available: 30-40 areas get 40-50 wind speeds
windgen_ons.loc[windgen_ons['AAA'].str.contains('LW') == False, 'AAA'] = windgen_ons['AAA'].str.replace('30-40', '40-50')

#One row for each AAA, for each hour
windgen_ons = pd.DataFrame(windgen_ons.groupby(['Y', 'C', 'RRR', 'AAA', 'G', 'timestamp'])['Val'].sum().reset_index())



#Create list with areas for which there is WS data
WS_areas = windspeeds.AAA.unique()

#Change names of areas that are not in WS data
forgotten =  windgen_ons.loc[(windgen_ons['AAA'].isin(WS_areas)==False)  & (windgen_ons['G'].str.contains('LW')==False), ]
forgotten.loc[forgotten['G'].str.contains('RG1'), 'AAA'] = forgotten.loc[forgotten['G'].str.contains('RG1'), 'RRR'] + '_VRE-ONS_RG1_20-30'
forgotten.loc[forgotten['G'].str.contains('RG2'), 'AAA'] = forgotten.loc[forgotten['G'].str.contains('RG2'), 'RRR'] + '_VRE-ONS_RG2_20-30'
forgotten.loc[forgotten['G'].str.contains('RG3'), 'AAA'] = forgotten.loc[forgotten['G'].str.contains('RG3'), 'RRR'] + '_VRE-ONS_RG3_20-30'
forgotten.loc[(forgotten['G'].str.contains('RG1') == False) & (forgotten['G'].str.contains('RG2') == False) & (forgotten['G'].str.contains('RG3') == False), 'AAA'] = \
forgotten.loc[(forgotten['G'].str.contains('RG1') == False) & (forgotten['G'].str.contains('RG2') == False) & (forgotten['G'].str.contains('RG3') == False), 'RRR'] + '_VRE-ONS_20-30'

#Create remaining dataframe
windgen_ons = windgen_ons.loc[windgen_ons['AAA'].isin(WS_areas)==True, ]

#Merge old and new dataframes
windgen_ons = [windgen_ons, forgotten]
windgen_ons = pd.concat(windgen_ons)
windgen_ons = pd.DataFrame(windgen_ons.groupby(['Y', 'C', 'RRR', 'AAA',  'timestamp'])['Val'].sum().reset_index())

C:\Users\marback\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Merge with wind speeds

In [11]:
df = pd.merge(windspeeds, windgen_ons[['AAA', 'timestamp', 'Val']], on = ['timestamp', 'AAA'], how = 'left')
df = df.loc[df['Val'].isnull()==False,] #Keep only wind turbines that produce (i.e. that exist)
df = df[['hour', 'timestamp', 'RRR', 'AAA', 'WS', 'Val']].reset_index(drop = True)

## Average WS Calculation 

In [12]:
df['W'] = df['WS'] * df['Val']
df_W = pd.DataFrame(df.groupby(['hour', 'timestamp', 'RRR'])['W'].sum().reset_index())
df_Val = pd.DataFrame(df.groupby(['hour', 'timestamp', 'RRR'])['Val'].sum().reset_index())

df_W['Val'] = df_Val['Val']
df_W['WS_avg'] = df_W['W'] / df_Val['Val']

#Add price column 
df_W = pd.merge(df_W, price_data[['timestamp', 'RRR', 'Price']], on = ['timestamp', 'RRR'])

#Make timestamp visible for hovering by writing it as string
df_W.loc[:, "timestamp"] = df_W.loc[:, "timestamp"].dt.strftime('%d %h %Y %H:%M')
df_scatter = df_W

## Region Aggregation

In [13]:
df_W['W_Price'] = df_W['Price'] * df_W['Val']
df_W['W_WS_avg'] = df_W['WS_avg'] * df_W['Val']
df_agg =  pd.DataFrame(df_W.groupby(['hour', 'timestamp'])['Val'].sum().reset_index())
df_agg['W_price'] = (pd.DataFrame(df_W.groupby(['hour', 'timestamp'])['W_Price'].sum().reset_index()))['W_Price']
df_agg['price_agg'] = df_agg['W_price'] / df_agg['Val']
df_agg['W_WS_avg'] = (pd.DataFrame(df_W.groupby(['hour', 'timestamp'])['W_WS_avg'].sum().reset_index() ))['W_WS_avg']
df_agg['W_WS_avg'] = df_agg['W_WS_avg'] / df_agg['Val']

# Making Output Directory

In [14]:
# Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')
    
# Make market folder
for market in markets:
    if not os.path.isdir('output/' + output + '/'+ market +'/' + scenario +'/' + year):
        os.makedirs('output/' + output + '/'+ market +'/' + scenario +'/' + year)

# Plotting

#### Create colour dictionary


In [15]:
color = len(regions)*color
color_dict = dict(zip(regions, color))

#### Create plot for aggregated regions

In [16]:
#Aggregated Plots
fig = make_subplots(rows=1, cols=3, subplot_titles = ['Production vs. Wind Speeds', 'Price vs. Wind Speeds', 'Production vs. Price'],shared_xaxes=False,vertical_spacing=0.05)
fig.add_trace(
            go.Scatter(x=df_agg['W_WS_avg'], y = df_agg['Val'] / 1000, mode = 'markers', marker=dict(color=color[0])),
            col=1, row=1
        )
fig.add_trace(
            go.Scatter(x=df_agg['W_WS_avg'], y = df_agg['price_agg'], mode = 'markers', marker=dict(color=color[1])),
            col=2, row=1
        )
fig.add_trace(
            go.Scatter(x=df_agg['Val'] / 1000, y = df_agg['price_agg'], mode = 'markers', marker=dict(color=color[2])),
            col=3, row=1
        )
fig.update_xaxes(title_text="Average Wind Speed (m/s)", row=1, col=1)
fig.update_yaxes(title_text="Aggregated Generation On-shore Wind (GWh)", row=1, col=1)
fig.update_xaxes(title_text="Average Wind Speed (m/s)", row=1, col=2)
fig.update_yaxes(title_text="Average Electricity Price (€/MWh)", row=1, col=2)
fig.update_xaxes(title_text="Average Generation On-shore Wind (GWh)", row=1, col=3)
fig.update_yaxes(title_text="Aggregated Electricity Price (€/MWh)", row=1, col=3)

fig.update_layout(title_text = 'Correlations of aggregated regions: ' + scenario + ', ' + year,   showlegend = False)
fig.write_html('output/' + output + '/'+ market +'/' + scenario + '/' + year + '/'+ output + '_' + scenario + '_' + year + '_aggregated.html', auto_open = True)


#### Function to make one plot for each region

In [17]:
def plot_per_region(display_regions, suffix):
    plot_titles = list(chain(*zip(display_regions, display_regions, display_regions) ) )
    fig = make_subplots(rows=len(display_regions), cols=3, subplot_titles = plot_titles,shared_xaxes=False,vertical_spacing=0.05)
    for country in display_regions:
        fig.add_trace(
            go.Scatter(name = country + ' ' + scenario + ', ' + year, \
                       x=df_scatter.loc[df_scatter['RRR'] == country, 'WS_avg'], \
                       y=df_scatter.loc[df_scatter['RRR'] == country, 'Val'], \
                       mode = 'markers', marker = dict(color= color_dict[country])),
            col=1, row=display_regions.index(country) +1
        )
        fig.add_trace(
            go.Scatter(name = country + ' ' + scenario + ', ' + year, \
                       x=df_scatter.loc[df_scatter['RRR'] == country, 'WS_avg'], \
                       y=df_scatter.loc[df_scatter['RRR'] == country, 'Price'], \
                       mode = 'markers', marker = dict(color= color_dict[country])),
            col=2, row=display_regions.index(country) + 1
        )
        fig.add_trace(
            go.Scatter(name = country + ' ' + scenario + ', ' + year, \
                       x=df_scatter.loc[df_scatter['RRR'] == country, 'Val'], \
                       y=df_scatter.loc[df_scatter['RRR'] == country, 'Price'], \
                       mode = 'markers', marker = dict(color= color_dict[country])),
            col=3, row=display_regions.index(country) + 1
        )

    for i in range(0, len(display_regions)):
        fig.update_xaxes(title_text="Wind Speed (m/s)", row=i + 1, col=1)
        fig.update_yaxes(title_text="Generation On-shore Wind (MWh)", row=i + 1, col=1)
    for i in range(len(display_regions)):
        fig.update_xaxes(title_text="Wind Speed (m/s)", row=i + 1, col=2)
        fig.update_yaxes(title_text="Electricity Price (€/MWh)", row=i + 1, col=2)
    for i in range(len(display_regions)):
        fig.update_xaxes(title_text="Generation On-shore Wind (MWh)", row=i + 1, col=3)
        fig.update_yaxes(title_text="Electricity Price (€/MWh)", row=i + 1, col=3)
    fig.update_layout(title_text = 'Wind speed vs. production (left), wind speeds vs. price (middle), production vs. price (right): ' + scenario + ', ' + year, height = 600*len(display_regions),  showlegend = False)


    fig.write_html('output/' + output + '/'+ market +'/' + scenario + '/' + year + '/'+ output + '_' + scenario + '_' + year + '_' + suffix + '.html', auto_open = True)    


#### Create plots per region

In [18]:
# If more than 8 countries need to be displayed, the html is split up (due to display constraints)
regions = list(regions)
if len(regions) <= 8:
    plot_per_region(regions, 'all')
if len(regions) > 8:
    regions_a = regions[0:8]
    plot_per_region(regions_a, 'a')
    regions_b = regions[8:16]
    plot_per_region(regions_b, 'b')
if len(regions) > 16:
    regions_c = regions[16:24]
    plot_per_region(regions_c, 'c') 
if len(regions) > 24:
    regions_d = regions[24:32]
    plot_per_region(regions_d, 'd') 